In [2]:
import mne
import pandas as pd
import numpy as np
import scipy.io
import scipy.signal as ss
from scipy.signal import hilbert,butter, lfilter,welch
from scipy.integrate import simps
from meegkit.asr import ASR
from meegkit.utils.matrix import sliding_window
from mne.time_frequency import psd_array_multitaper            
from neurodsp.utils.download import load_ndsp_data
sampling_rate = 250

def butterBandpass(data, lower_limit_filter, upper_limit_filter, sampling_rate, order=4):
    """
    This func is for filtering signal between lower and upper bounds
    the methods are used from scipy.signal lib
    """
    nyquist_coeff = 0.5 * sampling_rate
    low_frequences_filter = lower_limit_filter / nyquist_coeff
    high_frequences_filter = upper_limit_filter / nyquist_coeff
    numerator_filter, denominator_filter = butter(order, 
                                                  [low_frequences_filter, high_frequences_filter],
                                                  btype='band')
    # based on numinator and denominator the filter signal ...                                            )
    filtered_signal = lfilter(numerator_filter, denominator_filter, data)
    return filtered_signal

def applyArtifactSubspaceReconstruction(raw, sfreq=sampling_rate, cutoff=2.5, 
                                        blocksize=100, win_len=0.5,
                                        win_overlap=0.66, max_dropout_fraction=0.1,
                                        min_clean_fraction=0.25, name='asrfilter', method='euclid',
                                        estimator='scm'):
    """
    Goal: this function removes artifact specially those related to EOG noises
    """
    h, w = raw.shape
    
    # t = int(raw.shape[1] / sfreq)
    # take asr ....
    asr = ASR(sfreq=sfreq, cutoff=cutoff, blocksize=blocksize, win_len=win_len,
        win_overlap=win_overlap, max_dropout_fraction=max_dropout_fraction,
        min_clean_fraction=min_clean_fraction, name=name, method=method,
        estimator=estimator)   
    # method='euclid'
    # train_idx = np.arange(0 * sfreq, t * sfreq, dtype=int)
    
    # short description ....
    _, sample_mask = asr.fit(raw)
    
    # Apply filter using sliding (non-overlapping) windows
    # name X, Y
    # description
    #sliding_window function converts our 2-D data(number of channels by number of recorded samples) dataset to a 3-D function(number of channels by time in second by sampling rate) 
    X = sliding_window(raw, window=int(sfreq), step=int(sfreq)) 
    Y = np.zeros_like(X)
    for i in range(X.shape[1]):
        Y[:, i, :] = asr.transform(X[:, i, :])
   # reshape to (n_chans, n_times)
    clean = Y.reshape(h, -1)
    return clean

ModuleNotFoundError: No module named 'meegkit'

In [14]:
import numpy as np
num_subjects=np.array([1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,
                       30,31,32,33,36,37,38,39,40,42,43,44,45])

for ii in range(len(num_subjects)):
    data=np.loadtxt('eeg_data_'+str(num_subjects[ii])+'.txt') as data:       
    filter_signal = butterBandpass(data, 1, 49, 250)
    filter_asr_signal = applyArtifactSubspaceReconstruction(filter_signal)
    np.save('s'+str(hh[ii])+'_clean_nurun_recorded_dataset', filter_asr_signal)